In [9]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Sort_Vertices, Bounding_Box, Raster_Centroid, Raster_BBox, Weighted_Raster_Centroid

In [10]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

In [11]:
image_resolution = 500
res = image_resolution
scale_factor = image_resolution / res
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
image_array = np.array(img)
weight_array = np.zeros([image_resolution,image_resolution])
pixels = 0
for x in range(image_resolution):
    for y in range(image_resolution):
        pixel = image_array[x,y]
        weight_array[x,y] = np.mean(pixel) / 255

In [17]:
polygon = np.array([[10,10],[10,20],[20,20],[20,10]])
bbox = Raster_BBox(polygon, image_resolution)
rasterized = Scanline_Rasterize_Polygon(polygon,bbox,image_resolution)
polygon

array([[10, 10],
       [10, 20],
       [20, 20],
       [20, 10]])

: 

In [16]:
centroid = Weighted_Raster_Centroid(rasterized, weight_array)
centroid

[15.0, 15.0]

In [15]:
scale_factor = 10
pixel_count = 0
img_width = int(image_resolution / scale_factor)
offset = 0
for x in range(img_width):
    for y in range(img_width):
        center_rectangle((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor*.8, scale_factor*.8,'white')
offset = 0
for pixel in rasterized:
    x = pixel[0]
    y = pixel[1]
    center_rectangle((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor*.8, scale_factor*.8,'red')

for point in polygon:
    x = point[0]
    y = point[1]
    center_ellipse((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor / 2,'blue')

for point in bbox:
    x = point[0]
    y = point[1]
    center_ellipse((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor / 4,'black')


center_ellipse((centroid[0] + offset)*scale_factor, (centroid[1] + offset)*scale_factor, scale_factor / 4,'black')



print(img_width)
print(50**2)
img.save('centroid_debug.png')

50
2500
